## Data Preprocessing

In [23]:
import os
import pandas as pd
import numpy as np
import openpyxl
data_dir = 'data'

### Generating Courses Data

In [ ]:
courses = pd.read_csv(os.path.join(data_dir, 'courses.csv'))
courses.sort_values(by=['category', 'course name'], ascending=[False, True], inplace=True)
courses.reset_index(inplace=True, drop=True)
courses["course_id"] = courses.index
courses.rename(columns={"course name": "course_name"}, inplace=True)
courses = courses[["course_id", "course_name", "category"]]

In [ ]:
courses["course_id"] = courses.index
courses.rename(columns={"course name": "course_name"}, inplace=True)
courses = courses[["course_id", "course_name", "category"]]
courses["course_id"] += 1
courses.to_csv(os.path.join(data_dir, "courses.csv"), index=False)

In [ ]:
courses = pd.read_csv(os.path.join(data_dir, 'courses.csv'))
courses

### Combine all sheets in a single CSV file


In [54]:
# even sem 2023-24.xlsx file contains 33 sheets, first one is NPTEL
final = pd.DataFrame(columns=['email', 'name', 'uid', 'branch', 'sem', 'course'])
final

,email,name,uid,branch,sem,course


In [55]:
wb1 = openpyxl.load_workbook(os.path.join(data_dir, 'odd sem 2022-23.xlsx'))
wb2 = openpyxl.load_workbook(os.path.join(data_dir, 'even sem 2022-23.xlsx'))
wb3 = openpyxl.load_workbook(os.path.join(data_dir, 'odd sem 2023-24.xlsx'))
wb4 = openpyxl.load_workbook(os.path.join(data_dir, 'even sem 2023-24.xlsx'))

c1 = len(wb1.sheetnames)
c2 = len(wb2.sheetnames)
c3 = len(wb3.sheetnames)
c4 = len(wb4.sheetnames)

print(c1, c2, c3, c4)

20 21 27 33


In [56]:
for sheet_no in range(0, 20):  
    df = pd.read_excel(
        os.path.join(data_dir, "odd sem 2022-23.xlsx"), sheet_name=sheet_no
    )
    df.columns = ["email", "name", "uid", "class", "branch", "course", "sem"]
    if sheet_no == 0:
        df.drop(16, inplace=True)
    df.drop(columns=["class"], inplace=True)
    df = df[["email", "name", "uid", "branch", "sem", "course"]]
    final = pd.concat([final, df], ignore_index=True)

In [69]:
for sheet_no in range(0, 21):
    df = pd.read_excel(
        os.path.join(data_dir, "even sem 2022-23.xlsx"), sheet_name=sheet_no
    )
    if sheet_no == 11:
        df.drop(columns=[df.columns[-1]], inplace=True)
    df.columns = ["timestamp", "email", "name", "uid", "class", "branch", "course", "sem"]
    df.drop(columns=["timestamp", "class"], inplace=True)
    df = df[["email", "name", "uid", "branch", "sem", "course"]]
    final = pd.concat([final, df], ignore_index=True)

0 (15, 8)
1 (27, 8)
2 (35, 8)
3 (6, 8)
4 (43, 8)
5 (8, 8)
6 (40, 8)
7 (46, 8)
8 (122, 8)
9 (40, 8)
10 (42, 8)
11 (41, 8)
12 (17, 8)
13 (30, 8)
14 (71, 8)
15 (67, 8)
16 (159, 8)
17 (131, 8)
18 (140, 8)
19 (18, 8)
20 (25, 8)


In [ ]:
for sheet_no in range(0, 27):
    df = pd.read_excel(os.path.join(data_dir, 'odd sem 2023-24.xlsx'), sheet_name=sheet_no)
    df.columns = ['email', 'name', 'uid', 'class', 'branch', 'course', 'sem']
    df.drop(columns=['class'], inplace=True)
    df = df[['email', 'name', 'uid', 'branch', 'sem', 'course']]
    final = pd.concat([final, df], ignore_index=True)

In [ ]:
for sheet_no in range(1, 33): # skipping nptel sheet
    df = pd.read_excel(os.path.join(data_dir, 'even sem 2023-24.xlsx'), sheet_name=sheet_no)
    df.columns = ['email', 'name', 'uid', 'class', 'branch', 'course', 'sem']
    df.drop(columns=['class'], inplace=True)
    df = df[['email', 'name', 'uid', 'branch', 'sem', 'course']]
    final = pd.concat([final, df], ignore_index=True)

In [57]:
final.isnull().sum()

email     0
name      0
uid       0
branch    0
sem       0
course    0
dtype: int64

In [58]:
# print row with missing values
final[final.isnull().any(axis=1)]

,email,name,uid,branch,sem,course


In [59]:
final.shape

(1204, 6)

In [ ]:
final.to_csv(os.path.join(data_dir, 'allsem.csv'), index=False)

In [ ]:
final.sort_values(by=['course', 'sem', 'branch', 'uid'])

In [ ]:
normalized_course_names = courses['course_name'].tolist()
normalized_course_names

In [ ]:
import difflib

difflib.get_close_matches('Technology', normalized_course_names)

In [73]:
import numpy as np

def find_closet_match(test_str, list2check):
    scores = {}
    test_str = test_str.lower()
    for ii in list2check:
        cnt = 0
        if len(test_str)<=len(ii):
            str1, str2 = test_str, ii
        else:
            str1, str2 = ii, test_str
        for jj in range(len(str1)):
            cnt += 1 if str1[jj]==str2[jj] else 0
        scores[ii] = cnt
    scores_values = np.array(list(scores.values()))
    closest_match_idx = np.argmax(scores_values)
    # closest_match_idx = np.argsort(scores_values, axis=0, kind='quicksort')[-3:]
    closest_match = np.array(list(scores.keys()))[closest_match_idx]
    return closest_match, closest_match_idx

In [74]:
find_closet_match('Personality', [i.lower() for i in normalized_course_names])

('physical fitness', 15)